In [7]:
using Curiosity, Statistics, Plots, FileIO, BSON
using Random, StatsBase, RollingFunctions
using GVFHordes

const MCU = Curiosity.MountainCarUtils

Curiosity.MountainCarUtils

In [8]:
# Cumulant
struct Reward end

(c::Reward)(;kwargs...) = kwargs[:r]

struct ConstantDiscount{F}
    val::F
end
   
(d::ConstantDiscount)(;kwargs...) = d.val

In [9]:
# Environment 
function construct_env()
    normalized = true
    env = MountainCar(0.0, 0.0, normalized)
end


construct_env (generic function with 1 method)

In [10]:
# Agent
function construct_agent(numtilings, numtiles, lu_str, α, λ, ϵ, γ)
    obs_size = 2
    fc = Curiosity.SparseTileCoder(numtilings, numtiles, obs_size)
    feature_size = size(fc)

    lu = if lu_str == "ESARSA"
        ESARSA(lambda=λ, opt=Curiosity.Descent(α))
    elseif lu_str == "SARSA"
        SARSA(lambda=λ, opt=Curiosity.Descent(α))
    elseif lu_str == "TB"
        TB(lambda=λ, opt=Curiosity.Descent(α))
    else
        throw(ArgumentError("$(lu_str) Not a valid behaviour learning update"))
    end
#     (update, num_features, num_actions, num_demons, w_init)
    learner = LinearQLearner(lu, feature_size, 3, 1,0)
    exploration = EpsilonGreedy(ϵ)
    cumulant = Reward()
    discount = ConstantDiscount(γ)
    
    b_gvf = make_behaviour_gvf(learner, discount, fc, exploration)
    b_demons = Horde([b_gvf])

    Curiosity.PolicyLearner(learner, 
                            fc, 
                            exploration, 
                            discount, 
                            cumulant, 
                            zeros(2), 
                            0,
                            b_demons)
end

construct_agent (generic function with 1 method)

In [11]:
function make_behaviour_gvf(behaviour_learner, γ, fc, exploration_strategy)
    function b_π(state_constructor, learner, exploration_strategy; kwargs...)
        s = state_constructor(kwargs[:state_t])
        preds = learner(s)
        return exploration_strategy(preds)[kwargs[:action_t]]
    end
    GVF_policy = GVFParamFuncs.FunctionalPolicy((;kwargs...) -> b_π(fc, behaviour_learner, exploration_strategy; kwargs...))
    BehaviourGVF = GVF(GVFParamFuncs.RewardCumulant(), GVFParamFuncs.ConstantDiscount(γ), GVF_policy)
end

make_behaviour_gvf (generic function with 1 method)

In [12]:
function make_behaviour_gvf(behaviour_learner, γ, fc, exploration_strategy)
    function b_π(state_constructor, learner, exploration_strategy; kwargs...)
        s = state_constructor(kwargs[:state_t])
        preds = learner(s)
        return exploration_strategy(preds)[kwargs[:action_t]]
    end
    GVF_policy = GVFParamFuncs.FunctionalPolicy((;kwargs...) -> b_π(fc, behaviour_learner, exploration_strategy; kwargs...))
    BehaviourGVF = GVF(GVFParamFuncs.RewardCumulant(), GVFParamFuncs.ConstantDiscount(γ), GVF_policy)
end

Float32[0.58050555, 0.40556338]


LoadError: BoundsError: attempt to access 3-element Vector{Float64} at index [  [1  ]  =  1
  [3  ]  =  1
  [4  ]  =  1
  [5  ]  =  1
  [6  ]  =  1
  [8  ]  =  1
  [9  ]  =  1
  [10 ]  =  1]

In [26]:
plot(rollmean(ret, 100))

LoadError: 
	Bad window span (100) for length 0.


In [34]:
# Save the policy